In [ ]:
import csv
import pandas as pd

# FROM CARD OPEN TO LAST MONTH OF TXN CUSTOMER SURVIVAL
(before that used sas to find debit cards)

In [ ]:
file = open('all_debit_cards_base_30.12.21_270_days_gone.csv')
csv_data = csv.reader(file,delimiter = ';')

In [ ]:
lines = list(csv_data)
lines[0]

In [ ]:
min_open = 0
max_txn = 0
break_count = 0
break_months = []
alive_flg = 0

lines[0].extend(['BREAK_MONTHS', 'BREAK_COUNT', 'MIN_OPEN', 'MAX_TXN', 'ALIVE_FLG'])

for i in range(1, len(lines)):
           
    if lines[i][0] != lines[i-1][0]:
        
        break_count = 0
        break_months = []
        alive_flg = 0
        
        min_open = int(lines[i][8])
        
        if lines[i][17] != '':
            max_txn = int(lines[i][17])
        else:
            max_txn = int(lines[i][8])
    
    else:

        if int(lines[i][8]) > max_txn:
            break_months.append(int(round((int(lines[i][8]) - max_txn) / 30, 0)))
            break_count += 1
        
            
        if lines[i][17] != '' and int(lines[i][17]) > max_txn:
            max_txn = int(lines[i][17])
        
    
    if lines[i][-1] == '1':
        alive_flg = 1
        
    lines[i].append(break_months)
    lines[i].append(break_count)
    lines[i].append(min_open)
    lines[i].append(max_txn)
    lines[i].append(alive_flg)

In [ ]:
# закрывает ли карту до перерыва

min_open = 0
max_txn = 0
max_close = 0


#yes - zakryl, no - ne zakryl kartu
count_yes = []
count_no = []
count_same_m = []

for i in range(1, len(lines)):
           
    if lines[i][0] != lines[i-1][0]:
        
        
        min_open = int(lines[i][8])
                
        try:
            max_close = int(lines[i][10])
        except:
            max_close = 'Active'
        
        if lines[i][17] != '':
            max_txn = int(lines[i][17])        
    
    else:
                   
        if max_close != 'Active':
            
            if int(lines[i][8]) > max_close and int(lines[i][8]) > max_txn:
                count_yes.append(i)
            elif int(lines[i][8]) < max_close and int(lines[i][8]) > max_txn:
                count_no.append(i)
            elif int(lines[i][8]) == max_close and int(lines[i][8]) > max_txn:
                count_same_m.append(i)
                       
            if lines[i][10] == '':
                max_close = 'Active'
            elif int(lines[i][10]) > max_close:
                max_close = int(lines[i][10])
                
        if lines[i][17] != '' and int(lines[i][17]) > max_txn:
                    max_txn = int(lines[i][17])

In [ ]:
count_yes[:10]

In [ ]:
count_no[:10]

In [ ]:
df = pd.DataFrame(lines)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.to_csv('lines_all_client_cards_30.12.21_270_days_gone.csv', index = False)

## after that used SAS to keep only needed information

# SURVIVAL ANALYSIS FOR DEBIT CARD CUSTOMERS

In [ ]:
file = open('to_compute_hazards_2018.csv')
csv_data = csv.reader(file,delimiter = ';')
lines = list(csv_data)

In [ ]:
survived_to = []
for i in range(1,len(lines)):

    try:
        survived_to.append(int(lines[i][-2]))
    except:
        pass
    
gone_on = []
for i in range(1, len(lines)):
    try:
        gone_on.append(int(lines[i][-3]))
    except:
        pass

In [ ]:
survived_to_dict = {}
for x in range(200):
    count_x = 0
    for item in survived_to:
        if item>=x:
            count_x += 1
        else:
            pass
    survived_to_dict[int('{}'.format(x))] = count_x

In [ ]:
gone_on_dict = {}
for i in range(1, 200):
    gone_on_dict[int('{}'.format(i))] = gone_on.count(i)

In [ ]:
df = pd.DataFrame.from_dict(survived_to_dict, orient='index')
df.to_excel('survive_to.xlsx', sheet_name='survive_to')

df2 = pd.DataFrame.from_dict(gone_on_dict, orient='index')
df2.to_excel('gone_on.xlsx', sheet_name='gone_on')

# RETURN RATE

In [ ]:
# 22615 01.12.21

In [ ]:
file = open('to_compute_return_rate_all_client_cards_21.12.21_180_days_gone.csv')
csv_data = csv.reader(file,delimiter = ';')

In [ ]:
lines = list(csv_data)
lines[1]

In [ ]:
lines[0]

In [ ]:
returned_on = []
for i in range(1,len(lines)):
    if lines[i][1] != '[]':
        returned_on.extend(lines[i][1].replace('[','').replace(']','').split(', '))


In [ ]:
return_on_2 = list(map(lambda x: int(x), returned_on))

In [ ]:
return_set = set()
for item in return_on_2:
    return_set.add(item)
return_set = list(return_set)
return_set.sort()
    
return_dict = {}
for item in return_set:
    return_dict[item] = return_on_2.count(item)

In [ ]:
idet_since_gone = []

for i in range(1, len(lines)):
    if lines[i][1] == '[]' and lines[i][-1] == '0':
        idet_since_gone.append(int(round((22615 - int(lines[i][4])) / 30)))

In [ ]:
idet_s_zakrytiya = {}

for i in range(1,200):
    
    count_x = 0
    
    for x in return_on_2:
        if x > i:
            count_x += 1
    
    for y in idet_since_gone:
        if y > i:
            count_x += 1
    
    idet_s_zakrytiya[i] = count_x

In [ ]:
df = pd.DataFrame.from_dict(return_dict, orient='index')
df.to_excel('returned_on.xlsx', sheet_name='returned')

df2 = pd.DataFrame.from_dict(idet_s_zakrytiya, orient='index')
df2.to_excel('idet_s_zakrytiya.xlsx', sheet_name='idet')

In [ ]:
# to find how many months customers are gone before becoming active again
# need return rate file data

months_between = []

for i in range(1, len(lines)):
    if lines[i][1] != '[]':
        months_between.extend(lines[i][1].replace('[', '').replace(']','').split(', '))

In [ ]:
months_between_int = list(map(lambda x: int(x), months_between))

In [ ]:
months_between_int.sort()

In [ ]:
months_between_dict = {}

for i in range(183):
    months_between_dict[i] = months_between_int.count(i)

In [ ]:
df3 = pd.DataFrame.from_dict(months_between_dict, orient='index')
df3.to_excel('months_between_dict2.xlsx', sheet_name='months_between')

# SWITCH TO PREMIUM CARD

In [ ]:
file = open('gold_to_premium_card.csv')
csv_data = csv.reader(file,delimiter = ';')

In [ ]:
lines = list(csv_data)
lines[0]

In [ ]:
lines[0].extend(['CARD_COUNT', 'G_OR_P_FLAG', 'BREAK_MONTHS', 'BREAK_COUNT', 'MIN_OPEN', 'MAX_CLOSE'])

for i in range(1, len(lines)):
           
    if lines[i][0] != lines[i-1][0]:
        card_count = 1
        g_or_p_flag = []
        break_count = 0
        break_months = []
        min_open = 0
        max_close = 0
        
        if lines[i][10] == '' and lines[i][11] == 'Card OK':
            max_close = ''
        elif lines[i][10] == '':
            max_close = int(lines[i+1][8])
        else:
            max_close = int(lines[i][10])
        
        min_open = int(lines[i][8])
    
    else:
        card_count += 1
                        
        if max_close != '':
            
            if int(lines[i][8]) > max_close and lines[i][10] != '':
                break_months.append(int(round((int(lines[i][8]) - max_close) / 30, 0)))
                break_count += 1
                max_close = int(lines[i][10])
            elif lines[i][10] == '' and lines[i][11] == 'Card OK':
                max_close = ''
            elif lines[i][10] != '' and int(lines[i][10]) > max_close:
                max_close = int(lines[i][10])
            else:
                pass
    
    if 'PLATINUM' in lines[i][12] or 'INFINITE' in lines[i][12] or 'BLACK EDITION' in lines[i][12]:
        g_or_p_flag.append(1)
    else:
        g_or_p_flag.append(0)
    
    lines[i].append(card_count)
    lines[i].append(g_or_p_flag)
    lines[i].append(break_months)
    lines[i].append(break_count)
    lines[i].append(str(min_open))
    if max_close == '':
        lines[i].append(max_close)
    else:
        lines[i].append(str(max_close))

In [ ]:
df = pd.DataFrame(lines)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.to_csv('lines_g_or_p_flag.csv', index = False)

# MONTHS BETWEEN TXNS

In [ ]:
file = open('debit_client_transactions_by_months.csv', 'r')
csv_data = csv.reader(file, delimiter=';')

In [ ]:
lines = list(csv_data)

In [ ]:
lines[1]

In [ ]:
len(lines[1])

In [ ]:
some_list = []
count_no_txn = 0
i = 1

while lines[1][i] == '':
    i += 1
    
else:

    for x in range(i,183):
        print(x)
        if x == len(lines[1]) and lines[1][len(lines[1])] == '':
            pass

        elif lines[1][i] == '':
            count_no_txn += 1

        elif lines[1][i] == '1':

            if count_no_txn > 0:
                some_list.append(count_no_txn)

            count_no_txn = 0


In [ ]:
some_list

# FORECASTING NUMBER OF DEBIT CUSTOMERS

In [ ]:
# need total clients from to_compute_hazards_all_client_cards_21.12.21

ratio_dict = {}
for i in range(1,125):
    try:
        result = 1 - gone_on_dict[i] / (survived_to_dict[i] + gone_on_dict[i])
    except:
        result = 1
    ratio_dict[i] = result

In [ ]:
ratio_dict

In [ ]:
file = open('forecast_active_clients_nov_2021_(including_nov).csv')
csv_data = csv.reader(file,delimiter = ';')
lines = list(csv_data)

In [ ]:
len(lines)

In [ ]:
lines[0]

In [ ]:
status_0 = {}
status_1 = {}
status_2 = {}
status_3 = {}
status_4 = {}
status_5 = {}

n_months_active_dec = []
for item in lines:
    try:
        n_months_active_dec.append(int(item[-2]))
    except:
        pass

In [ ]:
for i in range(0,121):
    status_0[i] = n_months_active_dec.count(i)

In [ ]:
status_0

In [ ]:
status_1[1]= 10000*ratio_dict[1]
for i in range(2,122):
    status_1[i] = int(round(status_0[i-1] * ratio_dict[i-1]))

In [ ]:
status_2[1]= 7000*ratio_dict[1]
for i in range(2,123):
    status_2[i] = int(round(status_1[i-1] * ratio_dict[i-1]))

In [ ]:
status_3[1]= 9000*ratio_dict[1]
for i in range(2,124):
    status_3[i] = int(round(status_2[i-1] * ratio_dict[i-1]))

In [ ]:
status_4[1]= 9000*ratio_dict[1]
for i in range(2,125):
    status_4[i] = int(round(status_3[i-1] * ratio_dict[i-1]))

In [ ]:
status_5[1]= 9000*ratio_dict[1]
for i in range(2,126):
    status_5[i] = int(round(status_4[i-1] * ratio_dict[i-1]))

In [ ]:
df = pd.DataFrame.from_dict(status_0, orient='index')
df.to_excel('status_0.xlsx', sheet_name='nov')

df = pd.DataFrame.from_dict(status_1, orient='index')
df.to_excel('status_1.xlsx', sheet_name='dec')

df = pd.DataFrame.from_dict(status_2, orient='index')
df.to_excel('status_2.xlsx', sheet_name='jan')

df = pd.DataFrame.from_dict(status_3, orient='index')
df.to_excel('status_3.xlsx', sheet_name='feb')

df = pd.DataFrame.from_dict(status_4, orient='index')
df.to_excel('status_4.xlsx', sheet_name='mar')

df = pd.DataFrame.from_dict(status_5, orient='index')
df.to_excel('status_5.xlsx', sheet_name='apr')


In [ ]:
## NOT USED

fact_usage = []
for i in range(1,len(lines)):
    try:
        fact_usage.append(int(lines[i][-2]))
    except:
        fact_usage.append(0)

fact_usage_dict = {}
for x in range(max(fact_usage)+1):
    fact_usage_dict[int('{}'.format(x))] = fact_usage.count(x)
    

In [ ]:
## NOT USED

started_using_on = {}

temp = []
for x in range(1, len(lines)):
    temp.append(int(lines[x][-1]))

for i in range(max(temp)+1):
    started_using_on[int('{}'.format(i))] = temp.count(i)

In [ ]:
# SURVIVED TO (NTH MONTH WHEN CARD WAS SURELY ACTIVE)

df2 = pd.DataFrame.from_dict(survived_to_dict, orient='index')
df2.to_excel('survive_to.xlsx', sheet_name='survive_to')

In [ ]:
# GONE ON (NTH MONTH WHEN CARD WAS CLOSED)

df3 = pd.DataFrame.from_dict(gone_on_dict, orient='index')
df3.to_excel('gone_on.xlsx', sheet_name='gone_on')

In [ ]:
# FACT USAGE (FROM FIRST TO LAST TXN)

df4 = pd.DataFrame.from_dict(fact_usage_dict, orient='index')
df4.to_excel('fact_usage.xlsx', sheet_name='fact_usage')

In [ ]:
# STARTED USING (ON NTH MONTH FIRST TXN)

df5 = pd.DataFrame.from_dict(started_using_on, orient='index')
df5.to_excel('started_using_on.xlsx', sheet_name='started_on')

In [ ]:
#
#
# FOR ANNUAL SURVIVAL
#
#

file = open('annual_could_close_in_2021.csv')
csv_data = csv.reader(file,delimiter = ';')
lines = list(csv_data)

In [ ]:
lines[0]

In [ ]:
survived_to = []
for i in range(1,len(lines)):
    try:
        survived_to.append(int(lines[i][-1]))
    except:
        pass
    
gone_on = []
for i in range(1, len(lines)):
    try:
        gone_on.append(int(lines[i][-2]))
    except:
        pass

In [ ]:
survived_to_dict = {}
for x in range(121):
    count_x = 0
    for item in survived_to:
        if item>=x:
            count_x += 1
        else:
            pass
    survived_to_dict[int('{}'.format(x))] = count_x

In [ ]:
gone_on_dict = {}
for i in range(1, 121):
    gone_on_dict[int('{}'.format(i))] = gone_on.count(i)

In [ ]:
df2 = pd.DataFrame.from_dict(survived_to_dict, orient='index')
df2.to_excel('survive_to.xlsx', sheet_name='survive_to')

df3 = pd.DataFrame.from_dict(gone_on_dict, orient='index')
df3.to_excel('gone_on.xlsx', sheet_name='gone_on')